In [1]:
import argparse
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, ConcatDataset

from model import Model
from utils import *
from data import *
from train_baseline import *

import os
import json
import resource
import sys
import pickle

In [2]:
parser = create_arg_parser()
tgt_market = 't1'
src_markets = 's1-s2'

args = parser.parse_args(f'--tgt_market {tgt_market} --src_markets {src_markets}'.split()) #

if torch.cuda.is_available() and args.cuda:
    torch.cuda.set_device(0)
args.device = torch.device('cuda' if torch.cuda.is_available() and args.cuda else 'cpu')

In [ ]:
# load pretrained model
model_dir = 'checkpoints/t1_s1-s2_toytest.model'
id_bank_dir = 'checkpoints/t1_s1-s2_toytest.pickle'

valid_run = 'valid_t1_s1-s2_toytest.tsv'

with open(id_bank_dir, 'rb') as centralid_file:
    my_id_bank = pickle.load(centralid_file)

mymodel = Model(args, my_id_bank)
mymodel.load(model_dir)

In [ ]:
############
## Target Market data
############
tgt_train_data_dir = os.path.join(args.data_dir, args.tgt_market, 'train.tsv')
tgt_train_ratings = pd.read_csv(tgt_train_data_dir, sep='\t')

print(f'loading {tgt_train_data_dir}')
tgt_task_generator = TaskGenerator(tgt_train_ratings, my_id_bank)
print('loaded target data!')

tgt_valid_dataloader = tgt_task_generator.instance_a_market_valid_dataloader(args.tgt_market_valid, args.batch_size)
tgt_test_dataloader = tgt_task_generator.instance_a_market_valid_dataloader(args.tgt_market_test, args.batch_size)

In [7]:
valid_run_mf = mymodel.predict(tgt_valid_dataloader)

In [8]:
tgt_valid_qrel = read_qrel_file('DATA/t1/valid_qrel.tsv')
task_ov, task_ind = get_evaluations_final(valid_run_mf, tgt_valid_qrel)

{'P_5': 0.08112717834631072, 'P_10': 0.05076010381905822, 'P_20': 0.030867630700778644, 'recall_5': 0.40563589173155357, 'recall_10': 0.5076010381905821, 'recall_20': 0.6173526140155728, 'ndcg_cut_10': 0.33007476729750357, 'map_cut_10': 0.27484653730482433}


In [9]:
print(task_ov)

{'P_5': 0.08112717834631072, 'P_10': 0.05076010381905822, 'P_20': 0.030867630700778644, 'recall_5': 0.40563589173155357, 'recall_10': 0.5076010381905821, 'recall_20': 0.6173526140155728, 'ndcg_cut_10': 0.33007476729750357, 'map_cut_10': 0.27484653730482433}
